In [1]:
from django.conf import settings
import stripe
stripe.api_key = settings.STRIPE_ACCOUNT_SID
from backend.utility import *
import time

In [29]:
user = User.objects.first()
print(user)

Email=veyorokon@gmail.com


In [89]:
def find_items(search, invoice):
    search = [term.lower() for term in search]
    found = []
    for item in invoice.lines.data:
        description = item.description.lower()
        if all(term in description for term in search):
            #print(item)
            found.append(item)
    return found

def delete_invoice_item(itemID):
    item = stripe.InvoiceItem.retrieve(itemID)
    item.delete()
    
def calculate_management_fee(subtotal):
    subtotal /= 100 #Format from stripe
    fee = calculate_anorak_fee(subtotal) #imported from utility
    return(int(fee*100))

def prevent_duplicate_fees(managementFees):
    duplicateTotal = -1
    managementFee = None
    if len(managementFees) == 1:
        managementFee = managementFees[0]
        duplicateTotal = managementFee.amount
    elif len(managementFees) > 1:
        mgtFee = managementFees[0]
        duplicateTotal = mgtFee.amount
        duplicates = managementFees[1:]
        for duplicateFee in duplicates:
            delete_invoice_item(duplicateFee.id)
            duplicateTotal += duplicateFee.amount
        managementFee = mgtFee
        
    return managementFee, duplicateTotal
    
def sync_management_charge():
    invoice = user.stripe_customer.get_stripe_upcoming_invoice()
    customer = user.stripe_customer
    duplicateFees = find_items("Anorak Management Fee", invoice)
    managementCharge, duplicateTotal = prevent_duplicate_fees(duplicateFees)
    if duplicateTotal == -1:
        return create_management_charge(invoice)
    return update_management_charge(invoice, managementCharge, duplicateTotal)


def update_management_charge(invoice, managementCharge, feeTotal=0.0):
    feeID = managementCharge.id
    totalWithoutFees = float(invoice.subtotal-feeTotal)
    print(totalWithoutFees)
    fee = calculate_management_fee(totalWithoutFees)
    item = stripe.InvoiceItem.modify(
                feeID,
                amount = fee
            )
    return item


def create_management_charge(invoice):
    customer = user.stripe_customer
    timeNow = int(calendar.timegm(time.gmtime()))
    fee = calculate_management_fee(invoice.subtotal)
    item = stripe.InvoiceItem.create(
            customer=customer.stripe_customer_id, 
            amount=fee, 
            currency="usd", subscription=customer.stripe_subscription_id,
            description="Anorak Management Fee",
            period = {
            "start": timeNow,
            "end": get_first_day_next_month_epoch(),
          },
        )
    return item


In [97]:
#create_management_charge(invoice)
item = sync_management_charge()

6708.0


In [74]:
# If an invoice is deleted, it will add a duplicate "management fee" invoice item
search = [ "family"]
#check_duplicate_management_fees()
#find_items(search)

In [8]:
item

<InvoiceItem invoiceitem id=ii_1EEkRUGjkoMLHlzNZw2VBmy4 at 0x7f81411c4a40> JSON: {
  "amount": 250,
  "currency": "usd",
  "customer": "cus_Ee5GDE9rGJ7uW6",
  "date": 1552773204,
  "description": "Anorak Management Fee",
  "discountable": true,
  "id": "ii_1EEkRUGjkoMLHlzNZw2VBmy4",
  "invoice": null,
  "livemode": false,
  "metadata": {},
  "object": "invoiceitem",
  "period": {
    "end": 1554076800,
    "start": 1552773199
  },
  "plan": null,
  "proration": false,
  "quantity": 1,
  "subscription": "sub_Ee5GIgHDyjTmMk",
  "unit_amount": 250
}